In [46]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, classification_report

import pandas as pd

In [47]:
# label order double check
true_capsnet = np.load('log/result_massivekb_ytrue_capsnet.npy')
true_ap3 = np.load('log/result_massivekb_ytrue_ap3.npy')
true_ap3 = np.array(list(map(int, true_ap3)))
true_pepformer = np.load('log/result_massivekb_ytrue_PepFormer.npy')
true_deepmspeptide = np.load('log/result_massivekb_ytrue_deepmspeptide.npy')
true_deepmspeptide = np.array(list(map(int, true_deepmspeptide)))
true_dbydeep = np.load('log/result_massivekb_ytrue_dbydeep.npy')
true_dbydeep = np.array(list(map(int, true_dbydeep)))

print(list(map(len, [true_capsnet, true_ap3, true_pepformer, true_deepmspeptide, true_dbydeep])))
print(
    sum(true_dbydeep == true_ap3),
    sum(true_dbydeep == true_deepmspeptide), 
    sum(true_dbydeep == true_capsnet),
    sum(true_dbydeep == true_pepformer),
)

df_test = pd.read_csv('/data/2021_SJH_detectability/data_human/test.csv')
print(len(df_test.label.values==true_dbydeep))

# get probability of models
prob_capsnet = np.load('log/result_massivekb_yprob_capsnet.npy')
prob_capsnet = np.array(list(map(lambda x: round(x, 4), prob_capsnet)))
prob_ap3 = np.load('log/result_massivekb_yprob_ap3.npy')
prob_ap3 = np.array(list(map(lambda x: round(x, 4), prob_ap3)))
prob_pepformer = np.load('log/result_massivekb_yprob_PepFormer.npy')
prob_pepformer = np.array(list(map(lambda x: round(x, 4), prob_pepformer)))
prob_deepmspeptide = np.load('log/result_massivekb_yprob_deepmspeptide.npy')
prob_deepmspeptide = np.array(list(map(lambda x: round(x, 4), prob_deepmspeptide.reshape(-1))))
prob_dbydeep = np.load('log/result_massivekb_yprob_dbydeep.npy')
prob_dbydeep = np.array(list(map(lambda x: round(x, 4), prob_dbydeep.reshape(-1))))

df_test[['prob_dbydeep', 'prob_deepmspeptide', 'prob_capsnet', 'prob_pepformer', 'prob_ap3']] = np.array([
    prob_dbydeep, 
    prob_deepmspeptide, 
    prob_capsnet, 
    prob_pepformer, 
    prob_ap3]).T

pred_capsnet = np.load('log/result_massivekb_ypred_capsnet.npy')
pred_ap3 = np.load('log/result_massivekb_ypred_ap3.npy')
pred_ap3 = np.array(list(map(int, pred_ap3)))
pred_pepformer = np.load('log/result_massivekb_ypred_PepFormer.npy')
pred_deepmspeptide = np.load('log/result_massivekb_ypred_deepmspeptide.npy')
pred_dbydeep = np.load('log/result_massivekb_ypred_dbydeep.npy')

df_test[['pred_dbydeep', 'pred_deepmspeptide', 'pred_capsnet', 'pred_pepformer', 'pred_ap3']] = np.array([
    pred_dbydeep, 
    pred_deepmspeptide, 
    pred_capsnet, 
    pred_pepformer, 
    pred_ap3]).T

[170706, 170706, 170706, 170706, 170706]
170706 170706 170706 170706
170706


# Sequence identity : feasibility

In [49]:
df_test[df_test.cterm.apply(lambda x: x[3:7])=='LEEL'].head(2)

,peptide,nterm,cterm,miss1,miss2,label,prob_dbydeep,prob_deepmspeptide,prob_capsnet,prob_pepformer,prob_ap3,pred_dbydeep,pred_deepmspeptide,pred_capsnet,pred_pepformer,pred_ap3
15,SQSTQISQELEELR,HGDQMAQKSQSTQIS,SQELEELRAEQQRLK,ZZZZZZZZZZZZZZZ,ZZZZZZZZZZZZZZZ,True,0.9819,0.9394,0.9287,1.9078,0.985,1,1,1,1,1
2142,GFRVILEELK,ALGIEFLKGFRVILE,RVILEELKSEGRQCQ,IEFLKGFRVILEELK,ZZZZZZZZZZZZZZZ,True,0.4483,0.6466,0.5929,-0.0158,0.830,0,1,0,1,1


# Sequence identity : whole

In [71]:
percentile = 0.8

df_train = pd.read_csv('/data/2021_SJH_detectability/data_human/train.csv')
df_train = df_train.loc[df_train.label==True]

tmp1 = df_train.peptide.apply(lambda s: s[-int(len(s)*percentile):]).values
tmp2 = df_train.peptide.values
half_seqs = {k:v for k, v in zip(tmp2, tmp1)}

In [ ]:
result = dict()
for idx1, (pep, half_pep) in enumerate(half_seqs.items()):
    dic = {pep:[]}
    for idx, (s, label, dbydeep, deepmspeptide, capsnet, pepformer, pepformer_lab, ap3) in enumerate(df_test[['peptide', 'label', 'prob_dbydeep', 'prob_deepmspeptide', 'prob_capsnet', 'prob_pepformer', 'pred_pepformer', 'prob_ap3']].values):
        dbydeep, deepmspeptide, capsnet, pepformer, ap3 = list(map(lambda x: round(x, 2), [dbydeep, deepmspeptide, capsnet, pepformer, ap3]))
#         if pep == s:
#             dic[pep].append((s, half_pep, label, dbydeep, deepmspeptide, capsnet, pepformer, ap3, pepformer_lab))
        if len(s) <= len(pep) and half_pep in s and s[-len(half_pep):] != half_pep:  #  and pep not in s
            dic[pep].append((s, half_pep, label, dbydeep, deepmspeptide, capsnet, pepformer, ap3, pepformer_lab))

    if len(dic[pep]) >= 2:
        print(idx1)
        for k, v in dic.items():
            for v_ in v:
                print(v_)
        result[pep] = dic

312
('AIEINPDSAQPYKWR', 'AIEINPDSAQPYK', True, 0.93, 0.79, 0.86, 1.13, 0.77, 1)
('AIEINPDSAQPYKWRGK', 'AIEINPDSAQPYK', True, 0.24, 0.42, 0.33, -1.58, 0.27, 0)
('AIEINPDSAQPYKR', 'AIEINPDSAQPYK', True, 0.97, 0.94, 0.89, 1.4, 0.87, 1)
1207
('SCLHDELLFTVEREK', 'CLHDELLFTVER', True, 0.75, 0.78, 0.75, 0.52, 0.59, 1)
('SCLHDELLFTVEREKR', 'CLHDELLFTVER', False, 0.26, 0.06, 0.34, -0.78, 0.19, 0)
2088
('TKLEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.96, 0.96, 0.87, 1.0, 0.78, 1)
('LEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.97, 0.91, 0.92, 1.41, 0.78, 1)
('IKLEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.97, 0.92, 0.87, 1.1, 0.73, 1)
('LEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.97, 0.91, 0.92, 1.41, 0.78, 1)
2324
('TKLEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.96, 0.96, 0.87, 1.0, 0.78, 1)
('LEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.97, 0.91, 0.92, 1.41, 0.78, 1)
('IKLEQQVDDLEGSLEQEKK', 'LEQQVDDLEGSLEQEK', True, 0.97, 0.92, 0.87, 1.1, 0.73, 1)
('LEQQVDDLEGSLEQEKK', 'LEQQV

In [ ]:
result